# Python Database on Azure Blob by Paul Chao #

# Prepare Azure / Azure 的前置作業 #
- Please [register](https://azure.microsoft.com/en-us/free/) Azure / 請[註冊](https://azure.microsoft.com/zh-tw/free/) Azure
- Please [login](https://portal.azure.com) Azure / 請[登入](https://portal.azure.com) Azure 帳號

## Azure Blob / Azure Blob介紹 ##

"binary large object"

"Azure Blob storage is a service for storing large amounts of unstructured object data, such as text or binary data, that can be accessed from anywhere in the world via HTTP or HTTPS. You can use Blob storage to expose data publicly to the world, or to store application data privately.+

Common uses of Blob storage include:

- Serving images or documents directly to a browser.
- Storing files for distributed access.
- Streaming video and audio.
- Storing data for backup and restore, disaster recovery, and archiving.
- Storing data for analysis by an on-premises or Azure-hosted service."


## Install Environment
``` bash
$ python3 -m venv env
$ source env/bin/activate
$ pip install azure-storage-blob  (pip >= 9.0)
```

``` Python
$ Python
>>> import azure.storage.blob
>>>
```
即為成功


In [2]:
!pip install azure.storage.blob

  Using cached azure_storage_blob-0.37.1-py2.py3-none-any.whl


In [3]:
import azure.storage.blob
help(azure.storage.blob)

Help on package azure.storage.blob in azure.storage:

NAME
    azure.storage.blob

DESCRIPTION
    # -------------------------------------------------------------------------
    # Copyright (c) Microsoft.  All rights reserved.
    #
    # Licensed under the Apache License, Version 2.0 (the "License");
    # you may not use this file except in compliance with the License.
    # You may obtain a copy of the License at
    #   http://www.apache.org/licenses/LICENSE-2.0
    #
    # Unless required by applicable law or agreed to in writing, software
    # distributed under the License is distributed on an "AS IS" BASIS,
    # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    # See the License for the specific language governing permissions and
    # limitations under the License.
    # --------------------------------------------------------------------------

PACKAGE CONTENTS
    _constants
    _deserialization
    _download_chunking
    _encryption
    _error
 

In [20]:
from azure.storage.blob import BlockBlobService

![](https://docs.microsoft.com/en-us/azure/includes/media/storage-blob-concepts-include/blob1.png)

## Setup Blob / 設定 Azure Blob ##
老師會現場教學

## Azure Setup / Azure 設定 ##

In [21]:
import os
storage_account= ??your_storage_account??
storage_key= ??your_storage_key??
container_name= "test"
local_file= os.getcwd() + "\\google.csv"      
blob_name= "contacts.csv"

## Get Service / 取得服務 ## 

In [39]:
srv = BlockBlobService(account_name=storage_account, account_key=storage_key)
srv

## Create Container in Code / 在code裡建立容器 ##

In [44]:
srv.create_container(container_name)

True

## Write a File as a Blob / 把檔案寫入blob ##

In [22]:
srv.create_blob_from_path(
    container_name,
    blob_name,
    local_file
)

## List Blobs in Container / 把容器裡面的blob列出來 ##

In [48]:
list = srv.list_blobs(container_name)
for blob in list:
    print(blob.name)

contacts.csv


## Read Blob from Azure / 將 blob 讀回來 ##

In [25]:
srv.get_blob_to_path(container_name, blob_name, 'new.csv')

## Whole Program / 程式集成 ##

In [47]:
from azure.storage.blob import BlockBlobService
import os

storage_account= ??your_storage_account??
storage_key= ??your_storage_key??
container_name= "test"

def create_service():
    return BlockBlobService(account_name=storage_account, account_key=storage_key)

def save_file_to_blob(service, filepath, blobname):
    service.create_blob_from_path(container_name, blobname, filepath)

def list_file_from_blob(service, containername):
    list = service.list_blobs(containername)
    for blob in list:
        print(blob.name)

def read_file_from_blob(service, blobname, savedname):
    service.get_blob_to_path(container_name, blobname, savedname)



# main procedure     
blob_name= "contacts.csv"
local_file= os.getcwd() + "\\google.csv"      

# create service, container and test storage
try:
    srv = create_service()
    print(srv)
    srv.create_container(container_name)
    print("Container {} created.".format(container_name))
except:
    print("Creating service/container errors.")

try:
    list_file_from_blob(srv, container_name)
    save_file_to_blob(srv, os.getcwd() + "\\google.csv", blob_name)
    print("Save file successfully")
except:            
    print ("Error happends during saving")

try:
    read_file_from_blob(srv, blob_name, "new.csv")
except:            
    print ("Error happends during reading")


Container test created.
Save file successfully


## Assignment 小作業 - 資料庫雲端備份 ##

試著將之前的 sqlite3 database 的代碼改良，將google.csv檔案轉換到資料庫後，將資料庫檔案寫到 Azure blob 上，再將雲端的資料庫檔案讀回來後，從裡面印出內容。

可參考之前的資料庫代碼修改。


## 參考答案 ##

In [53]:
import sqlite3
import os
import pandas as pd
from azure.storage.blob import BlockBlobService

# Database
upload_database_name = "sqlite3_test3.db"

def feed_database(pd_df, database_name):
    with sqlite3.connect(database_name) as db:
        pd_df.to_sql('member_info', con = db, if_exists='replace')

def read_database(database_name):
    with sqlite3.connect(database_name) as db:
        read_name = pd.read_sql_query('select * from member_info', con = db)
        print(read_name.head())
        print("\n")
        print(read_name.describe())

# Cloud
storage_account= ??your_storage_account??
storage_key= ??your_storage_key??
container_name= "database"
download_database_name = "sqlite3_test4.db"

def create_service():
    return BlockBlobService(account_name=storage_account, account_key=storage_key)

def save_file_to_blob(service, filepath, blobname):
    service.create_blob_from_path(container_name, blobname, filepath)

def list_file_from_blob(service, containername):
    list = service.list_blobs(containername)
    for blob in list:
        print(blob.name)

def read_file_from_blob(service, blobname, savedname):
    service.get_blob_to_path(container_name, blobname, savedname)

# main procedure
print("preparing data ...\n")
df = pd.read_csv("google.csv", encoding = "ISO-8859-1")
new_df = df.loc[1:10,['Name', 'Family Name', 'Birthday','Occupation']].set_index('Name')

# write to database to-be-uploaded
print("Write data to database ...\n")
feed_database(new_df, upload_database_name)

# get service and create container
print("Get Azure Storage service and create container\n")
srv = create_service()
srv.create_container(container_name)

# do upload
print("Writing data to cloud ... result ={}\n".format(
        save_file_to_blob(srv, upload_database_name, "sqlite_cloud.db"))
     )

# list 
print("List blobs on cloud, container={}\n".format(container_name))
list_file_from_blob(srv, container_name)

# do download
print("Download database from cloud ...\n")
print(read_file_from_blob(srv, "sqlite_cloud.db", download_database_name))

# read newly downloaded database
print("Read data from downloaded database...\n")
read_database(download_database_name)

preparing data ...

Write data to database ...



C:\Users\User\Anaconda3\lib\site-packages\pandas\core\generic.py:1345: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


Get Azure Storage service and create container

Writing data to cloud ... result =None

List blobs on cloud, container=database

sqlite_cloud.db
Download database from cloud ...

None
Read data from downloaded database...

      Name Family Name    Birthday       Occupation
0      Bob       Chang  1997-01-02         Engineer
1  Charles       Chong  1997-01-03         Engineer
2    Daisy        Wang  1997-01-04  Senior Engineer
3    Ellie         Lai  1997-01-05          Manager
4    Fanny        Wang  1997-01-06              CEO


       Name Family Name    Birthday Occupation
count    10          10          10         10
unique   10           8          10          5
top     Bob       Chang  1997-01-09   Engineer
freq      1           2           1          5


### Note : Storage Explorer ###
Download Azure Storage [Explorer](https://azure.microsoft.com/en-us/features/storage-explorer/)

# 工作坊實作專案 - 自製超強雲端備份工具 #
還記得今天我們所教的 traverse directory 嗎? 試試看作一個上傳程式(upload.py)，把指定目錄中所有的檔案全部放入乾淨的 container。

再作一個還原程式 (download.py)，執行之後可以把剛剛上傳的所有檔案(含目錄)，還原到目前的目錄裡。

這樣我們就完成了一個自製的雲端備份工具，可以在各台電腦上將目錄結構傳到雲端備份，並且能夠在其他電腦上還原備份。


## 在此之前你還需要知道一點小技巧 ##

### 1. sub-directory of blob / blob的子目錄問題 ###

In [56]:
srv

In [57]:
srv.create_blob_from_path("test", "test2/contacts.csv", "google.csv")

In [60]:
def create_service():
    return BlockBlobService(account_name=storage_account, account_key=storage_key)

def save_file_to_blob(service, filepath, blobname):
    print("blobname={}".format(blobname))
    service.create_blob_from_path(container_name, blobname, filepath)

def list_file_from_blob(service, containername):
    list = service.list_blobs(containername)
    for blob in list:
        print(blob.name)

def read_file_from_blob(service, blobname, savedname):
    service.get_blob_to_path(container_name, blobname, savedname)
    
srv = create_service()
srv.create_container("test")

save_file_to_blob(srv, "google.csv", "contacts.csv")
save_file_to_blob(srv, "google.csv","test1/contacts.csv")
list_file_from_blob(srv, "test")
read_file_from_blob(srv, "test1/contacts.csv", "google2.csv")

blobname=contacts.csv
blobname=test1/contacts.csv


### 2. Recursively create sub-directory locally / 遞迴建立子目錄問題 ###

In [1]:
import os
help(os.makedirs)

Help on function makedirs in module os:

makedirs(name, mode=511, exist_ok=False)
    makedirs(name [, mode=0o777][, exist_ok=False])
    
    Super-mkdir; create a leaf directory and all intermediate ones.  Works like
    mkdir, except that any intermediate path segment (not just the rightmost)
    will be created if it does not exist. If the target directory already
    exists, raise an OSError if exist_ok is False. Otherwise no exception is
    raised.  This is recursive.



In [2]:
os.makedirs("images/testdir")

In [3]:
os.makedirs("images/testdir")

FileExistsError: [WinError 183] 當檔案已存在時，無法建立該檔案。: 'images/testdir'

In [4]:
os.makedirs("images/testdir", exist_ok=True)

In [5]:
os.makedirs("images/testdir", exist_ok=True)

### 3. 最後的小提醒，避免收費以及耗費頻寬 : 
- 可以加入過濾器 (只傳特殊副檔名) 
- 上傳檔案時，注意上傳目錄裡面的檔案數與大小，建議作個測試目錄來玩玩即可
- 下載還原時，可以用 input() 指定寫入目錄，避免現行工作目錄被弄亂

# 讓我們開始動手作吧! #

## Advanced Topic ##

In [27]:
dir(write_blob_service)

['MAX_BLOCK_SIZE',
 'MAX_CHUNK_GET_SIZE',
 'MAX_SINGLE_GET_SIZE',
 'MAX_SINGLE_PUT_SIZE',
 'MIN_LARGE_BLOCK_UPLOAD_THRESHOLD',
 '_USER_AGENT_STRING',
 '_X_MS_VERSION',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__metaclass__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply_host',
 '_copy_blob',
 '_get_blob',
 '_get_host_locations',
 '_httpclient',
 '_lease_blob_impl',
 '_lease_container_impl',
 '_list_blobs',
 '_list_containers',
 '_perform_request',
 '_put_blob',
 '_put_block',
 '_put_block_list',
 '_update_user_agent_string',
 'abort_copy_blob',
 'account_key',
 'account_name',
 'acquire_blob_lease',
 'acquire_container_lease',
 'authentication',
 'blob_type',
 'break_blob_lease',
 'break_container

用 Context Manager 來包裝